# Practical Project 

The aim of this project is to solve various forms of the portfolio optimization problem through Julia/computer software. This program template will provide hints/template code for you to focus on the optimization formulation part.

As usual, we first need to load a few packages in Julia.

# Part I - Compulsory Task

In [29]:
# Load the JuMP related packages and several solvers
# ECOS - for solving SOCP problems
# Juniper & Ipopt - for solving MI-NLP problems
using JuMP, Juniper, ECOS, Ipopt
# Load the data/file processing related packages
using CSV, Glob, DataFrames, Statistics
# Load the Plot package for illustrating the solution
using Plots
# Load the custom functions for benchmarking  
include("./reusablefunc.jl");

#### Data Preprocessing
The first step is to load the raw data into the memory. The following codes are provided for you to help you get started. 

In [30]:
# Change "subgroup1" to other names according to the subgroup you are assigned.
path_subgroup = "./ftec_project_subgroup2/" 
files = glob( "*_train.csv", path_subgroup );
dfs = DataFrame.( CSV.File.( files ) );

In [31]:
T = 800; n = length(dfs);
stocks_retur = zeros(T,n);
for i = 1:n
    # compute the realized return R_i(t)
    stocks_retur[:,i] = (dfs[i].close-dfs[i].open) ./ dfs[i].open;
end
names_stocks = [ dfs[i].Name[1] for i in 1:n ];

## Task 4: Warm-up Exercise

For part (a) of this task, you have to plot the return of 3-4 stocks over time. An example is provided for you as follows. Use "Insert" -> "Insert Cell Below/After" if you want to keep the plots together in one place.

In [32]:
stock_id = 1; # You may change this stock_id to different numbers from 1 to 20
plot( dfs[stock_id].date, stocks_retur[:,stock_id] , label = dfs[stock_id].Name[1], title = dfs[stock_id].Name[1]*"'s return over time" )
savefig("stock1.png");

In [33]:
stock_id = 8; # You may change this stock_id to different numbers from 1 to 20
plot( dfs[stock_id].date, stocks_retur[:,stock_id] , label = dfs[stock_id].Name[1], title = dfs[stock_id].Name[1]*"'s return over time" )
savefig("stock2.png");

In [34]:
stock_id = 15; # You may change this stock_id to different numbers from 1 to 20
plot( dfs[stock_id].date, stocks_retur[:,stock_id] , label = dfs[stock_id].Name[1], title = dfs[stock_id].Name[1]*"'s return over time" )
savefig("stock3.png");

In [35]:
stock_id = 20; # You may change this stock_id to different numbers from 1 to 20
plot( dfs[stock_id].date, stocks_retur[:,stock_id] , label = dfs[stock_id].Name[1], title = dfs[stock_id].Name[1]*"'s return over time" )
savefig("stock4.png");

For part (b) of this task, we need to estimate the expected return $\hat{r}_i$ and covariance $\hat\rho_{ij}$. Notice that these terms are given by (1.6) in the project specification. For your convenience, they have been calculated as follows:

In [36]:
# calculate r_i and Sigma
bar_R = [ mean( stocks_retur[:,i] ) for i in 1:length(dfs) ];
Sigma = [ mean( (stocks_retur[:,i].-bar_R[i]).*(stocks_retur[:,j].-bar_R[j]) ) for i=1:n, j=1:n ]; 

where "bar_R" is a $20$-dimensional vector containing the expected return $\hat{r}$ for the stocks; and "Sigma" is the $20 \times 20$ matrix of the covariance. 

## Task 5: Closed Form Solution to (1.1)

This task computes the optimal portfolio using the closed form solution derived in Task 1. Here are a few hints of useful syntax in computing the optimal solution:

- To compute the inverse of a square matrix, e.g., "$\texttt{Sigma}$", it can be done by 
$$\texttt{Sigma^-1}$$
- To create a (column) vector of all ones of $n$-dimensional. you may use 
$$\texttt{ones(n)}.$$ 
- In your closed form solution, you may need encounter something such as ${\bf 1}^\top {S} {b}$ for some $n \times n$ square matrix ${S}$, and $n$-dimensional vector ${b}$. The above expression can be computed in Julia as
$$\texttt{ones(n)'*S*b}$$
where $\texttt{ones(n)'}$ has denoted the transpose of the vector $\texttt{ones(n)}$. 

In [37]:
# your code here
B = 20;
B1=30;
B2=40;
one_vec = ones(20);
Rd = 1.01 * sum(bar_R);
Rd1 = 1.1*sum(bar_R);
Rd2 = 1.5*sum(bar_R);
r0 = bar_R'*Sigma^-1*bar_R;
r1 = one_vec'*Sigma^-1*bar_R;
r2 = one_vec'*Sigma^-1*one_vec;
portfolio_opt = Sigma^-1 *((r0*B-r1*Rd)*one_vec+(r2*Rd-r1*B)*bar_R)/(r0*r2-r1^2);

In [38]:
portfolio_opt1 = Sigma^-1 *((r0*B1-r1*Rd)*one_vec+(r2*Rd-r1*B1)*bar_R)/(r0*r2-r1^2);
portfolio_opt2 = Sigma^-1 *((r0*B2-r1*Rd)*one_vec+(r2*Rd-r1*B2)*bar_R)/(r0*r2-r1^2);
portfolio_opt3 = Sigma^-1 *((r0*B-r1*Rd1)*one_vec+(r2*Rd1-r1*B)*bar_R)/(r0*r2-r1^2);
portfolio_opt4 = Sigma^-1 *((r0*B-r1*Rd2)*one_vec+(r2*Rd2-r1*B)*bar_R)/(r0*r2-r1^2);

Suppose that $\texttt{portfolio_opt}$ has been created as a 20-dimensional vector of the optimal portfolio. The following helper code should plot the comparison of the portfolio for you. 

In [39]:
plot( names_stocks, portfolio_opt, labels = "portfolio", xticks = :all )
# you may adjust the scale factor "1000" to scale up/down the expected return to make it comparable with 
# the value of the portfolio (*for improved visualization only*).
plot!( names_stocks, 3000*bar_R , labels = "(scaled) expected return") 
plot!( names_stocks, 3000*[Sigma[i,i] for i in 1:n], labels = "(scaled) variance" )
savefig("5-normal.png");

In [40]:
plot( names_stocks, portfolio_opt, labels = "portfolio-B=20", xticks = :all )
plot!(names_stocks, portfolio_opt1, labels = "portfolio-B=30", xticks = :all )
plot!(names_stocks, portfolio_opt2, labels = "portfolio-B=40", xticks = :all )
# you may adjust the scale factor "1000" to scale up/down the expected return to make it comparable with 
# the value of the portfolio (*for improved visualization only*).
plot!( names_stocks, 3000*bar_R , labels = "(scaled) expected return") 
plot!( names_stocks, 3000*[Sigma[i,i] for i in 1:n], labels = "(scaled) variance" )
savefig("5-exp-B.png")

In [41]:
plot( names_stocks, portfolio_opt, labels = "portfolio-Rd=1.01", xticks = :all )
plot!(names_stocks, portfolio_opt3, labels = "portfolio-Rd=1.1", xticks = :all )
plot!(names_stocks, portfolio_opt4, labels = "portfolio-Rd=1.5", xticks = :all )
# you may adjust the scale factor "1000" to scale up/down the expected return to make it comparable with 
# the value of the portfolio (*for improved visualization only*).
plot!( names_stocks, 3000*bar_R , labels = "(scaled) expected return") 
plot!( names_stocks, 3000*[Sigma[i,i] for i in 1:n], labels = "(scaled) variance" )
savefig("5-exp-Rd.png")

## Task 6 (a): Mixed Integer Programming Solution

For this task, we shall implement the MI-NLP problem. As usual, we have to define the optimization object and specify a few parameters, as follows. The solver we are going to apply is "Juniper".

In [42]:
# the following code specifies the constants as described in the problem
M = 20; B = 20; c = 2; w = 1*ones(n); Rd = 1.01*sum( w.*bar_R ); 

# the following code setup the JuMP model with the right solver
nl_solver = optimizer_with_attributes(Ipopt.Optimizer, "print_level"=>0)
optimizer = Juniper.Optimizer
model = Model(optimizer_with_attributes(optimizer, "nl_solver"=>nl_solver, "atol"=>1e-10));

You can program the MI-NLP problem in the following cell and solve it. Here are a few hints that maybe useful.

- To be compatible with the helper codes in the latter section, please call the decision variable for the portfolio by "x_mip". 
- You may use for-loop to specify a large number of constraints. 
- To model constraint given in the form of 
$$ \sum_{i=1}^n x_i y_i \geq r $$
withe the $n$-dimensional vectors $x$, $y$. You may do so by

$$ \texttt{@constraint(model, sum( x .* y ) >= r)} $$

where $\texttt{.*}$ denotes an "element-wise" product 
- To minimize quadratic function of the form
$$ f(x) = x^\top \Sigma x $$

we first notice that 

$$ f(x) = ( \Sigma^{1/2} x )^\top ( \Sigma^{1/2} x ) $$

and the above can be modeled in JuMP by

$$ \texttt{@variable( model, y[1:n] )}$$
$$ \texttt{@constraint( model, y .== sqrt(Sigma)*x );}$$ 
$$ \texttt{@NLobjective( model, Min, sum(y[i]^2 for i in 1:n) )} $$

Notice that we shall use "$\texttt{NLObjective}$ to specify that the objective function is nonlinear.

In [43]:
# your code here
n = 20;
@variable(model, k[1:n]);
@variable(model, y[1:n], Bin);
@variable(model, x_mip[1:n]);
@constraint(model, k .== sqrt(Sigma)*(x_mip+w));
@constraint(model, sum((x_mip+w) .* bar_R) >= Rd);
@constraint(model, sum(x_mip+y.*c)<=B);
@constraint(model, x_mip .>= -M.*y);
@constraint(model, x_mip .<= M.*y);
@constraint(model, x_mip .>= -w);
@NLobjective(model, Min, sum(k[i]^2 for i in 1:n));
optimize!(model);

atol              : 1.0e-10
nl_solver         : MathOptInterface.OptimizerWithAttributes(Ipopt.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[RawParameter("print_level")=>0])
feasibility_pump  : false
log_levels        : Symbol[:Options, :Table, :Info]

#Variables: 60
#IntBinVar: 20
#Constraints: 82
#Linear Constraints: 82
#Quadratic Constraints: 0
#NonLinear Constraints: 0
Obj Sense: Min

Start values are not feasible.
Status of relaxation: LOCALLY_SOLVED
Time for relaxation: 0.06200003623962402
Relaxation Obj: 0.004248858348537353

 ONodes   CLevel          Incumbent                   BestBound            Gap    Time   Restarts  GainGap  
    2       2                 -                          0.0                -     0.5       0         -     
    3       3                 -                          0.0                -     0.5       -       77.5%   
    4       4                 -                          0.0                -     0.5       -       78.3%   
    5 

## Task 6(b): SOCP Solution

For this task, we shall implement the SOCP program formulated in Task 3. As usual, we have to define the optimization object and specify a few parameters, as follows. The solver we are going to apply is "ECOS".

In [44]:
# specify the problem parameters
M = 20; B = 20; a = 2; w = ones(n); Rd = 1.01*sum( w.*bar_R ); 
# specify the JuMP model with ECOS as the optimizer
m_socp = Model( ECOS.Optimizer );

You may program the SOCP problem into JuMP as follows. Again, for convenience, you may name the decision variable of the portfolio as "x_socp". 

Further, wou may find that some constraints are similar to the MI-NLP from the previous task. However, when you "copy-and-paste" those code, don't forget to change the model name and the variable name. Here are some hints on modeling the second order cone constraints:

- To model a SOC constraint given in the form

$$ \| Ax + d \| \leq c^\top x + d, $$

you can use

$$ \texttt{ @constraint( m_socp, [c'*x + d; A*x + d] in SecondOrderCone() ) } $$

Essentially, "$\texttt{[c'*x + d; A*x + d]}$" defines a vector whose first element describes the RHS of the SOC constraint, the the remaining elements describe the vector found inside the norm of the SOC constraint. 

In [45]:
# your code here
using LinearAlgebra
n = 20;
a1 = zeros(n+1,n+1);
b1 = [Rd;zeros(n)];
c1 = [0;bar_R];
d1 = bar_R'*w;
a2 = [zeros(n) sqrt(a).*Array{Float64}(I,n,n)];
b2 = 1/(2*sqrt(a)).*ones(n);
d2 = sqrt(B+n/(4*a));
a3 = [zeros(n) sqrt(Sigma)];
b3 = sqrt(Sigma)*w;
c3 = [1;zeros(n)];
@variable(m_socp,x_socp[1:n+1]);
@constraint(m_socp,[c1'*x_socp+d1;a1*x_socp+b1] in SecondOrderCone());
@constraint(m_socp,[d2;a2*x_socp+b2] in SecondOrderCone());
@constraint(m_socp,[c3'*x_socp;a3*x_socp+b3] in SecondOrderCone());
@constraint(m_socp,[i=2:n+1],-M <= x_socp[i] <= M);
@constraint(m_socp,[i=2:n+1],x_socp[i] >= -w[i-1]);
@objective(m_socp, Min, x_socp[1]);
optimize!(m_socp)


ECOS 2.0.5 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+00  -8.984e+02  +1e+03  8e-02  6e-01  1e+00  2e+01    ---    ---    1  1  - |  -  - 
 1  +1.005e+00  -4.956e+02  +6e+02  4e-02  5e-01  3e+00  1e+01  0.5795  3e-01   1  1  1 |  0  0
 2  +7.074e-01  -1.868e+02  +2e+02  2e-02  2e-01  1e+00  4e+00  0.6350  5e-02   1  1  1 |  0  0
 3  +1.114e+00  -1.236e+02  +2e+02  1e-02  2e-02  2e+00  2e+00  0.8912  6e-01   1  2  2 |  0  0
 4  +1.214e-01  -1.409e+01  +2e+01  1e-03  5e-03  2e-01  3e-01  0.9091  4e-02   2  2  2 |  0  0
 5  +1.102e-01  -4.652e-01  +8e-01  5e-05  1e-03  9e-03  1e-02  0.9890  3e-02   2  2  2 |  0  0
 6  +8.429e-02  -2.818e-02  +2e-01  1e-05  2e-04  2e-03  3e-03  0.8194  2e-02   1  1  1 |  0  0
 7  +8.578e-02  -1.586e-02  +1e-01  9e-06  2e-04  2e-03  2e-03  0.2615  6e-01   1  1  1 |  0  0
 8  +7.437e-02  +3.386e-02  +6e-02  4e-06  8e-

## Task 6(c): Plotting the portfolios found

Given that you have programmed and executed the optimization problems correctly, the following helper code shall plot the portfolios nicely for you. 

In [46]:
plot( names_stocks, portfolio_opt, labels = "unconstrained portfolio" )
plot!( names_stocks, JuMP.value.(x_mip + w), labels = "MI-NLP")
plot!( names_stocks, JuMP.value.(x_socp[2:n+1] + w), labels = "SOCP")
plot!( names_stocks, 3000*bar_R, labels ="(Scaled) Expected Return")
plot!( names_stocks, 3000*[Sigma[i,i] for i in 1:n], labels ="(Scaled) Variance")
savefig("three.png")

## Task 7: Evaluating the Solution on Testing Set

Again, provided that you have programmed and executed the optimization problems correctly, the following helper code shall compute the Sharpe ratio and other benchmarks for you.

In [47]:
sharpe_IP = sharpe_ratio( path_subgroup, JuMP.value.(x_mip), ones(n), 2 );

Sharpe Ratio = 0.07743046905112376, Return = 0.00436591035529453, Tx Cost = 28, Portfo Value = 11.16058819372345

In [48]:
sharpe_SOCP = sharpe_ratio( path_subgroup, JuMP.value.(x_socp[2:n+1]), ones(n) , 2 );

Sharpe Ratio = 0.062327854157761015, Return = 0.0034451790827156716, Tx Cost = 40, Portfo Value = 11.226746195677338

In [49]:
sharpe_Opt = sharpe_ratio( path_subgroup, portfolio_opt, zeros(n), 2 );

Sharpe Ratio = 0.10687408899641704, Return = 0.010452885714344958, Tx Cost = 40, Portfo Value = 20.000000000000004

# Part II - Competitive Task

In the compulsory task, we shall implement a projected gradient method (using a constant step size) for the approximated Portfolio optimization problem. 

We shall consider the full portfolio optimization problem. For this, let us first load the stock data with the following helper code.

In [50]:
# load the full data set!
files = glob( "*_train.csv", "./ftec_project_files/");
dfs = DataFrame.( CSV.File.( files ) );
T = 800; n = length(dfs);
stocks_retur_full = zeros(T,n);
for i = 1:n
    # compute the realized return R_i(t)
    stocks_retur_full[:,i] = (dfs[i].close-dfs[i].open) ./ dfs[i].open;
end
names_stocks_full = [ dfs[i].Name[1] for i in 1:n ];
# calculate r_i and Sigma
bar_R_full = [ mean( stocks_retur_full[:,i] ) for i in 1:length(dfs) ];
Sigma_full = [ mean( (stocks_retur_full[:,i].-bar_R_full[i]).*(stocks_retur_full[:,j].-bar_R_full[j]) ) for i=1:n, j=1:n ]; 

Notice that "bar_R_full" is the expected return for all the $n=471$ stocks considered, and "Sigma_full" is the $471 \times 471$ covariance matrix for them. 

## Task 8: Implementing a Customized Solver for Approximated problem

You shall write a few helper functions to compute the objective values, the gradient vector, the projection into the box constraint, etc.. to help you with implementing the customized solver. Some useful syntax are as follows:

- For a nonlinear function $h(z)$ (such as the Huber function) on a scalar $z$. Suppose that $x$ is an $n$-dimensional vector, to create the vector 

$$ [h(x)]_i = h(x_i) $$

you may use the syntax

$$ \texttt{h.(x)} $$

where the "." after "h" broadcasts the function to every elements of the vector. 
- Note that the objective function should be dependent on "x", "w", "a", "bar_R_full", "Sigma_full", "upsilon", "zeta". 

In [51]:
# your code/functions here
B = 20;
Rd = 1.01*sum(bar_R_full);
function calc_gamma(delta)
    delta = 0.01;
    L = a/delta + max(diag(Sigma_full));
    gamma = 1/L;
    return gamma;
end

function gd(x,w,a,lambda,upsilon,delta)
    @assert length(x)==n;
    grad = Sigma_full*x+Sigma_full*w - lambda.*bar_R_full+upsilon.*ones(n);
    for i=1:n
        if abs(x[i])> delta
            grad[i] += sign(x[i])*upsilon*a;
        else
            grad[i] += upsilon*(a/delta)*x[i];
        end
    end
    return grad;
end

function obj_v(x,w,a,lambda,upsilon,delta)
    @assert length(x)==n;
    obj = 1/2*((x+w)'*Sigma_full*(x+w))+lambda*(Rd-bar_R_full'*x-bar_R_full'*w)+(upsilon*sum(x)-upsilon*B);
    for i=1:n
        if abs(x[i])>delta
            obj += upsilon*(a*abs(x[i])-a*delta/2);
        else
            obj += upsilon*(a/(2*delta)*x[i]^2);
        end
    end
    return obj;
end
            

obj_v (generic function with 1 method)

You may program the iterative algorithm of your choice as follows. 

In [52]:
# set the parameters as specified by the problem
M = 20; w = 1*ones(n); a = 1; upsilon = 10; lambda = 34280; delta = 0.01;

# initialize the algorithm
x_custom = zeros(n); 
store_obj = []
push!(store_obj, obj_v(x_custom,w,a,lambda,upsilon,delta) ) # replace ".." with the function you wrote for computing the objective val.

# calculate l and u - your code here (should be a simple formula)
u = M.*ones(n);
l = -w;

for iteration_no = 1 : 50000 # feel free to adjust the number of iterations run here.
    # your code here
    gamma = 1/(iteration_no+1);
    x_custom -= gamma* gd(x_custom,w,a,lambda,upsilon,delta);
    for i=1:n
        if x_custom[i] > u[i]
            x_custom[i] = u[i];
        elseif x_custom[i] < l[i]
            x_custom[i] = l[i];
        end
    end
    push!(store_obj, obj_v(x_custom,w,a,lambda,upsilon,delta) ) # replace ".." with the function you wrote for computing the objective val.
end

In [53]:
plot( store_obj, title = "objective value") # plot the trajectory of the optimization algorithm
savefig("obj.png")

Apply the post-processing step as specified in the project.

In [54]:
x_custom_pp = copy( x_custom )
x_custom_pp[ abs.(x_custom_pp) .< delta ] .= 0;

The following code computes the sharpe ratio, return, transaction cost, total cost of portfolio which will be used to calculate your score for the competitive task!

In [55]:
sharpe_PPGD = sharpe_ratio( "./ftec_project_files/", x_custom_pp, ones(n), 2 )

Sharpe Ratio = 0.09294117457434602, Return = 1.3015641214060631, Tx Cost = 424, Portfo Value = 2981.296055217791

0.09294117457434602

It may also help to visualize the portfolio with the following code:

In [56]:
plot( names_stocks_full, x_custom_pp + w, labels = "gradient descent" )
plot!( names_stocks_full, 1000*bar_R_full, labels ="(Scaled) Expected Return")
plot!( names_stocks_full, 1000*[Sigma_full[i,i] for i in 1:n], labels ="(Scaled) Variance")
savefig("competitive.png")